In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import implicit
import scipy.sparse as sp

c:\Users\benja\Documents\Universidad\2025-2\Recomendadores\Proyecto\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Metricas

In [2]:
animes = pd.read_csv('data/animes.csv')

In [3]:
df_train = pd.read_csv(
    "train", sep=",", names=["userid", "itemid", "rating"], header=None
)

df_train.rating = [1 if x >= 5 else 0 for x in df_train.rating]

df_train.head()

,userid,itemid,rating
0,2052,12445.0,1
1,5141,34599.0,1
2,3340,37510.0,1
3,588,853.0,1
4,4822,27775.0,1


In [4]:
df_test = pd.read_csv(
    "test", sep=",", names=["userid", "itemid", "rating"], header=None
)

df_test.head()

,userid,itemid,rating
0,18887,37450.0,10
1,8831,32379.0,2
2,37283,36882.0,9
3,35602,10490.0,9
4,39042,NaN,6


In [5]:
item_interaction_counts = df_train['itemid'].value_counts()
user_count = df_train['userid'].nunique()
item_popularity = (item_interaction_counts / user_count).to_dict()
metadata = animes[['uid', 'genre']]
item_categories: dict[int, set[str | None]] = {}
for row in metadata.itertuples():
    item_categories[int(row[1]) if row[1].is_integer() else row[1]] = set(map(lambda i: i.strip(), row[2].split(','))) if isinstance(row[2], str) else set()

In [6]:
# drop nan items
df_train = df_train.dropna(subset=['itemid'])

In [7]:
user_items = {}
itemset = set()

for row in df_train.itertuples():
    if row[1] not in user_items:
        user_items[row[1]] = []

    user_items[row[1]].append(row[2])
    itemset.add(row[2])

itemset = np.sort(list(itemset))

sparse_matrix = np.zeros((len(user_items), len(itemset)))

for i, items in enumerate(user_items.values()):
    sparse_matrix[i] = np.isin(itemset, items, assume_unique=True).astype(int)

matrix = sp.csr_matrix(sparse_matrix.T)

user_item_matrix = matrix.T.tocsr()

In [ ]:
from metrics import precision_at_k, ndcg_at_k, novelty, diversity, recall_at_k, average_precision_at_k

In [19]:
user2row = {user_id: matrix_row for matrix_row, user_id in enumerate(user_items.keys())}
row2user = {matrix_row: user_id for user_id, matrix_row in user2row.items()}

item2col = {item_id: matrix_col for matrix_col, item_id in enumerate(itemset)}
col2item = {matrix_col: item_id for item_id, matrix_col in item2col.items()}

In [10]:
user_items_test = {}

for row in df_test.itertuples():
    if row[1] not in user_items_test:
        user_items_test[row[1]] = []

    user_items_test[row[1]].append(row[2])

In [11]:
model_als = implicit.als.AlternatingLeastSquares(factors=300)
model_als.fit(user_item_matrix, show_progress=False)

c:\Users\benja\Documents\Universidad\2025-2\Recomendadores\Proyecto\.venv\Lib\site-packages\implicit\cpu\als.py:95: RuntimeWarning: OpenBLAS is configured to use 12 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


In [ ]:
def evaluate_model(n, user_groups, delta=0.05):
    """
    Evalúa el modelo calculando métricas globales y de fairness.
    
    Args:
        n (int): El 'k' para las métricas @k.
        user_groups (dict): {user_id: 'group_label'}
        delta (float): Umbral de disparidad para considerar "sesgado".
    """
    
    group_scores = {
        'recall': {},  
        'precision': {} 
    }
    
    all_recalls = []
    all_aps = []
    all_ndcgs = []
    all_novelties = []
    all_diversities = []
    all_precisions = []
    
    for user_id in user_items_test.keys():
        
        # 1. Obtener datos del usuario
        if user_id not in user_groups:
            continue
        
        if user_id not in user2row:
            continue
            
        group = user_groups[user_id]
        truth_items = user_items_test.get(user_id, set())
        
        # 2. Obtener recomendaciones y vector de relevancia
        user_row = user2row[user_id]
        
        rec = model_als.recommend(user_row, user_item_matrix[user_row], n)[0]
        rec = np.array([col2item[col] for col in rec])
        
        rel_vector = np.isin(rec, truth_items, assume_unique=True).astype(int)

        # 3. Calcular métricas individuales
        user_recall = recall_at_k(rel_vector, n)
        user_precision = precision_at_k(rel_vector, n)
        user_ap = average_precision_at_k(rel_vector, n)
        user_ndcg = ndcg_at_k(rel_vector, n)
        user_novelty = novelty(rec, item_popularity)
        user_diversity = diversity(rec, n, item_categories)

        # 4. Almacenar para métricas globales
        all_recalls.append(user_recall)
        all_precisions.append(user_precision)
        all_aps.append(user_ap)
        all_ndcgs.append(user_ndcg)
        all_novelties.append(user_novelty)
        all_diversities.append(user_diversity)

        # 5. Almacenar métricas de fairness por grupo
        if group not in group_scores['recall']:
            group_scores['recall'][group] = []
            group_scores['precision'][group] = []
            
        group_scores['recall'][group].append(user_recall)
        group_scores['precision'][group].append(user_precision)

    # --- 6. Calcular Métricas Promedio (Globales) ---
    
    metrics_global = {
        'mean_recall': np.mean(all_recalls) if all_recalls else 0.0,
        'mean_precision': np.mean(all_precisions) if all_precisions else 0.0,
        'mean_ap (MAP)': np.mean(all_aps) if all_aps else 0.0,
        'mean_ndcg': np.mean(all_ndcgs) if all_ndcgs else 0.0,
        'mean_novelty': np.mean(all_novelties) if all_novelties else 0.0,
        'mean_diversity': np.mean(all_diversities) if all_diversities else 0.0,
        'num_users_evaluated': len(all_recalls)
    }
    
    # --- 7. Calcular Métricas de Fairness (Disparidad) ---
    
    fairness_report = {
        'delta_threshold': delta,
        'is_biased_recall': 0,
        'is_biased_precision': 0,
        'group_averages': {},
        'disparity_reports': []
    }

    avg_recall_group = {g: np.mean(s) for g, s in group_scores['recall'].items() if s}
    avg_precision_group = {g: np.mean(s) for g, s in group_scores['precision'].items() if s}
    
    fairness_report['group_averages'] = {
        g: {
            'recall (Cobertura)': avg_recall_group.get(g, 0.0),
            'precision (Tasa Aceptación)': avg_precision_group.get(g, 0.0),
            'count': len(group_scores['recall'].get(g, []))
        } for g in avg_recall_group.keys()
    }

    # Comparar pares de grupos
    group_names = list(avg_recall_group.keys())
    for i in range(len(group_names)):
        for j in range(i + 1, len(group_names)):
            g_a = group_names[i]
            g_b = group_names[j]
            
            r_a, r_b = avg_recall_group[g_a], avg_recall_group[g_b]
            abs_disp_recall = abs(r_a - r_b)
            
            p_a, p_b = avg_precision_group[g_a], avg_precision_group[g_b]
            abs_disp_precision = abs(p_a - p_b)

            pair_report = {
                'pair': (g_a, g_b),
                'recall_disparity': abs_disp_recall,
                'precision_disparity': abs_disp_precision,
                'biased_recall (Cobertura)': 1 if abs_disp_recall > delta else 0,
                'biased_precision (Tasa Aceptación)': 1 if abs_disp_precision > delta else 0
            }
            fairness_report['disparity_reports'].append(pair_report)
            
            if abs_disp_recall > delta:
                fairness_report['is_biased_recall'] = 1
            if abs_disp_precision > delta:
                fairness_report['is_biased_precision'] = 1
    
    return metrics_global, fairness_report

In [13]:
profiles = pd.read_csv('data/profiles.csv')
profiles.head()

,profile,gender,birthday,favorites_anime,link
0,DesolatePsyche,Male,"Oct 2, 1994","['33352', '25013', '5530', '33674', '1482', '2...",https://myanimelist.net/profile/DesolatePsyche
1,baekbeans,Female,"Nov 10, 2000","['11061', '31964', '853', '20583', '918', '925...",https://myanimelist.net/profile/baekbeans
2,skrn,NaN,NaN,"['918', '2904', '11741', '17074', '23273', '32...",https://myanimelist.net/profile/skrn
3,edgewalker00,Male,Sep 5,"['5680', '849', '2904', '3588', '37349']",https://myanimelist.net/profile/edgewalker00
4,aManOfCulture99,Male,"Oct 30, 1999","['4181', '7791', '9617', '5680', '2167', '4382...",https://myanimelist.net/profile/aManOfCulture99


In [14]:
reviews = pd.read_csv("data/reviews.csv")

# Assign a unique id to each username
unique_users = reviews["username"].unique()
user_mapping = {user: i for i, user in enumerate(unique_users)}
reviews["user_id"] = reviews["username"].map(user_mapping)

In [15]:
profiles["user_id"] = profiles["profile"].map(user_mapping)

In [20]:
import json

user_groups_map = {}
# user_groups_map id a sexo
for row in profiles.itertuples():
    user_id = row.user_id
    user_groups_map[user_id] = row.gender

K = 10
DELTA_FAIRNESS = 0.05

global_metrics, fairness_results = evaluate_model(K, user_groups_map, DELTA_FAIRNESS)

print("--- Métricas Globales de Evaluación ---")
print(json.dumps(global_metrics, indent=2))

print("\n--- Reporte de Fairness (Disparidad de Grupo) ---")
print(json.dumps(fairness_results, indent=2))

print(f"\nRecall Global: {global_metrics['mean_recall']:.4f}")
print(f"MAP Global: {global_metrics['mean_ap (MAP)']:.4f}")
print(f"¿Es sesgado (Recall)?: {fairness_results['is_biased_recall']}")
print(f"¿Es sesgado (Precision)?: {fairness_results['is_biased_precision']}")

--- Métricas Globales de Evaluación ---
{
  "mean_recall": 0.02736883066346715,
  "mean_precision": 0.0030352162915883002,
  "mean_ap (MAP)": 0.012203687061005953,
  "mean_ndcg": 0.015884514594982833,
  "mean_novelty": 8.479939905283517,
  "mean_diversity": 0.909071042154162,
  "num_users_evaluated": 15419
}

--- Reporte de Fairness (Disparidad de Grupo) ---
{
  "delta_threshold": 0.05,
  "is_biased_recall": 0,
  "is_biased_precision": 0,
  "group_averages": {
    "NaN": {
      "recall (Cobertura)": 0.023988615572270788,
      "precision (Tasa Aceptaci\u00f3n)": 0.002663142915226672,
      "count": 4919
    },
    "Female": {
      "recall (Cobertura)": 0.02261985145172181,
      "precision (Tasa Aceptaci\u00f3n)": 0.0023970290344361915,
      "count": 2962
    },
    "Male": {
      "recall (Cobertura)": 0.03133864649466433,
      "precision (Tasa Aceptaci\u00f3n)": 0.003525597730649737,
      "count": 7403
    },
    "Non-Binary": {
      "recall (Cobertura)": 0.037037037037037035,
